In [1]:
import sys
sys.path = ["/Users/bernhard/Development/hcplot/"] + sys.path

In [2]:
from hcplot.highcharts import Figure, PlotOptions, DrillDown, DrillUpButton, SeriesOptions, Series, SeriesData, Title, Legend, Chart, XAxis, DataLabels, \
    Position, SvgTheme, SvgButtonTheme, ButtonStates
from hcplot.highcharts.series import ColumnOptions


- Highsoft software products (www.highcharts.com) are not free for commercial and Governmental use.
  (see https://shop.highsoft.com/faq#Non-Commercial-0 for conditions for Non-commercial use).
- This product includes color specifications and designs developed by Cynthia Brewer
  http://colorbrewer.org
- D3 category colors and the interpolateRGB algorithm have been taken over from https://d3js.org/,
  Copyright 2010-2017 Mike Bostock



In [3]:
f = Figure(
    title=Title(text="Graph"), 
    chart=Chart(type_="column"), 
    xAxis=XAxis(type_="category"), 
    legend=Legend(enabled=False), 
    plotOptions=PlotOptions(       
        series=ColumnOptions(
            borderWidth=0, 
            dataLabels=DataLabels(enabled=True)
        )
    ), 
    series=[
        ColumnOptions(
            name="Things", 
            colorByPoint=True,
            data=[
                SeriesData(name='Dieren', y=5, drilldown='animals'),
                SeriesData(name='Fruit', y=2, drilldown='fruits'),
                SeriesData(name='Auto\'s', y=4)
            ]
        )
    ],
    drilldown=DrillDown(
        drillUpButton=DrillUpButton(
            relativeTo='spacingBox',
            position=Position(x=0, y=0),
            theme=SvgButtonTheme(
                fill='white',
                stroke_width=1,
                stroke='silver',
                r=0,
                states=ButtonStates(
                    hover=SvgTheme(
                        fill='#a4edba'
                    ),
                    select=SvgTheme(
                        stroke='#039',
                        fill='#a4edba'
                    )
                )
            )
        ),
        series=[
            ColumnOptions(
                id_='animals',
                data=[
                    ['Katten', 4],
                    ['Honden', 2],
                    ['Koeien', 1],
                    ['Schapen', 2],
                    ['Varkens', 1]
                ]
            ), 
            ColumnOptions(
                id_='fruits',
                data=[
                    ['Appels', 4],
                    ['Sinaasappels', 2]
                ]
            )
        ]
    )
)

In [4]:
f.toDict()

{'chart': {'type': 'column'},
 'drilldown': {'drillUpButton': {'position': {'x': 0.0, 'y': 0.0},
   'relativeTo': 'spacingBox',
   'theme': {'fill': 'white',
    'r': 0,
    'states': {'hover': {'fill': '#a4edba'},
     'select': {'fill': '#a4edba', 'stroke': '#039'}},
    'stroke': 'silver',
    'strokewidth': 1}},
  'series': [{'data': [['Katten', 4],
     ['Honden', 2],
     ['Koeien', 1],
     ['Schapen', 2],
     ['Varkens', 1]],
    'id': 'animals'},
   {'data': [['Appels', 4], ['Sinaasappels', 2]], 'id': 'fruits'}]},
 'legend': {'enabled': False},
 'plotOptions': {'series': {'borderWidth': 0,
   'dataLabels': {'enabled': True}}},
 'series': [{'colorByPoint': True,
   'data': [{'drilldown': 'animals', 'name': 'Dieren', 'y': 5.0},
    {'drilldown': 'fruits', 'name': 'Fruit', 'y': 2.0},
    {'name': "Auto's", 'y': 4.0}],
   'name': 'Things'}],
 'title': {'text': 'Graph'},
 'xAxis': {'type': 'category'}}

In [5]:
print(f.toJson())

{"xAxis": {"type": "category"}, "series": [{"colorByPoint": true, "name": "Things", "data": [{"drilldown": "animals", "name": "Dieren", "y": 5.0}, {"drilldown": "fruits", "name": "Fruit", "y": 2.0}, {"name": "Auto's", "y": 4.0}]}], "chart": {"type": "column"}, "plotOptions": {"series": {"borderWidth": 0, "dataLabels": {"enabled": true}}}, "title": {"text": "Graph"}, "drilldown": {"series": [{"id": "animals", "data": [["Katten", 4], ["Honden", 2], ["Koeien", 1], ["Schapen", 2], ["Varkens", 1]]}, {"id": "fruits", "data": [["Appels", 4], ["Sinaasappels", 2]]}], "drillUpButton": {"position": {"x": 0.0, "y": 0.0}, "relativeTo": "spacingBox", "theme": {"strokewidth": 1, "states": {"hover": {"fill": "#a4edba"}, "select": {"fill": "#a4edba", "stroke": "#039"}}, "stroke": "silver", "fill": "white", "r": 0}}}, "legend": {"enabled": false}}


In [288]:
from hcplot.highcharts.base import HCBase
from hcplot.highcharts.types import Color
from traitlets import Float, Instance, List, Tuple, TraitType, Undefined

from traitlets import HasTraits
import json


class HCBase(HasTraits):

    def handleList(self, value):
        result = []
        for el in value:
            if isinstance(el, (tuple, list)):
                result.append(self.handleList(el))
            else:
                print("->", el, hasattr(el, "class_traits"))
                result.append(el)
        return result
    
    def toDict(self):
        result = {}
        for key, cls in self.class_traits().items():
            value = getattr(self, key)
            print(key, cls, value)
            if value is not None:
                if key[-1] == "_":
                    key = key[:-1]
                key = key.replace("_", "")

                if isinstance(value, HCBase):
                    print(1)
                    result[key] = value.toDict()
                elif hasattr(cls, "conv"):
                    print(2)
                    result[key] = cls.conv(value)
                else:
                    print(3)
                    if isinstance(value, (tuple, list)):
                        print(4)
                        if len(value) > 0:
                            print(5)
                            result[key] = []
                            for el in value:
                                if hasattr(el, "class_traits"):
                                    for el2 in el.class_traits().items():
                                        
                                    print(8, el.class_traits().items())
                                    result[key].append(el.conv())
                                elif isinstance(value[0], HCBase):
                                    print(9)
                                    result[key].append(value.toDict())
                                else:
                                    print(10)
                                    result[key].append(el)
                    elif isinstance(value, dict):
                        print(6)
                        if len(value) > 0:
                            result[key] = {k: v.toDict() for k, v in value.items()}
                    else:
                        print(7)
                        result[key] = value
        return result

    def toJson(self, indent=None):
        return json.dumps(self.toDict(), indent=indent)


class LinearGradient(HCBase):
    x1 = Float(0.0, allow_none=False)
    x2 = Float(0.0, allow_none=False)
    y1 = Float(1.0, allow_none=False)
    y2 = Float(1.0, allow_none=False)


class RadialGradient(HCBase):
    cx = Float(0.0, allow_none=False)
    cy = Float(0.0, allow_none=False)
    r = Float(1.0, allow_none=False)


class Gradient(TraitType):
    default_value = 0
    info_text = 'LinearGradient or RadialGradient'

    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        super(__class__, self).__init__(default_value=default_value,          # noqa F821
                                        allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        if type is None or type(value) in [LinearGradient, RadialGradient]:
            return value
        else:
            self.error(obj, value)

class GradientStop(HasTraits):
    stop = Float(None, allow_none=True)
    color = Color(None, allow_none= True)

    
class _GradientStop(TraitType):
    default_value = None
    info_text = 'GradientStop'

    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        super(__class__, self).__init__(default_value=default_value,          # noqa F821
                                        allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        if (value is None) or (type(value) == GradientStop):
            return value
        else:
            self.error(obj, value)

    @staticmethod
    def conv(value):
        return (value.stop, Color.conv(value.color))


class FillColor(HCBase):
    linearGradient = Instance(LinearGradient, allow_none=True)
    radialGradient = Instance(RadialGradient, allow_none=True)
    stops = List(_GradientStop(None, allow_none=True))


In [289]:
fc = FillColor(
    linearGradient=LinearGradient(x1=0, x2=0, y1=200, y2=500),
    stops=[GradientStop(stop=0.0, color="Green"), GradientStop(stop=1.0, color="Red")]
)

In [290]:
fc.toDict()

radialGradient <traitlets.traitlets.Instance object at 0x10ea62f98> None
stops <traitlets.traitlets.List object at 0x10ea490b8> [<__main__.GradientStop object at 0x10e93d240>, <__main__.GradientStop object at 0x10e93d908>]
3
4
5
8 dict_items([('stop', <traitlets.traitlets.Float object at 0x10ea62f28>), ('color', <hcplot.highcharts.types.Color object at 0x10ea62f60>)])


AttributeError: 'GradientStop' object has no attribute 'conv'

In [76]:
class X(HCBase):
    color = Color(None, allow_none=True)

In [77]:
x = X(color="Blue")

In [78]:
x.toDict()

==> color <hcplot.highcharts.types.Color object at 0x10e7da5f8>


{'color': 'rgba(0,0,255,1.000)'}

In [110]:

test = (0.0, Color((0,0,255)))

In [112]:
isinstance(test[1], Color)

True

In [ ]:
fc.__class__